In [35]:
#ESTABLISHING CONNECTION WITH SQL SERVER
import mysql.connector


mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  #database='joins'
  
)

print(mydb)
mycursor = mydb.cursor(buffered=True)

In [36]:
#CREATING WAREHOUSE DATABASE
mycursor.execute("CREATE DATABASE Warehousedatabase")

In [37]:
#COMMAND TO USE WAREHOUSE DATABASE CREATED EARLIER
mycursor.execute("USE Warehousedatabase")

In [38]:
#CREATING TABLE NAMED Warehouses
mycursor.execute("""
CREATE TABLE Warehouses (
   Code INTEGER NOT NULL PRIMARY KEY,
   Location VARCHAR(255) NOT NULL,
   Capacity INTEGER NOT NULL
)
""")

#CREATING TABLE NAMED Boxes
mycursor.execute("""
CREATE TABLE Boxes (
    Code CHAR(4) NOT NULL PRIMARY KEY,
    Contents VARCHAR(255) NOT NULL,
    Value REAL NOT NULL,
    Warehouse INTEGER NOT NULL,
    FOREIGN KEY (Warehouse) REFERENCES Warehouses(Code)
) ENGINE=INNODB;
""")

#INSERTING DATA INTO TABLE Warehouses
mycursor.execute("INSERT INTO Warehouses(Code,Location,Capacity) VALUES(1,'Chicago',3)")
mycursor.execute("INSERT INTO Warehouses(Code,Location,Capacity) VALUES(2,'Chicago',4)")
mycursor.execute("INSERT INTO Warehouses(Code,Location,Capacity) VALUES(3,'New York',7)")
mycursor.execute("INSERT INTO Warehouses(Code,Location,Capacity) VALUES(4,'Los Angeles',2)")
mycursor.execute("INSERT INTO Warehouses(Code,Location,Capacity) VALUES(5,'San Francisco',8)")

#INSERTING DATA INTO TABLE Boxes
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('0MN7','Rocks',180,3)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4H8P','Rocks',250,1)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4RT3','Scissors',190,4)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('7G3H','Rocks',200,1)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8JN6','Papers',75,1)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8Y6U','Papers',50,3)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('9J6F','Papers',175,2)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('LL08','Rocks',140,4)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P0H6','Scissors',125,1)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P2T6','Scissors',150,2)")
mycursor.execute("INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('TU55','Papers',90,5)")


In [39]:
#Query to Select all warehouses
mycursor.execute ("SELECT * FROM warehouses")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+---------------+------------+
|   Code | Location      |   Capacity |
|--------+---------------+------------|
|      1 | Chicago       |          3 |
|      2 | Chicago       |          4 |
|      3 | New York      |          7 |
|      4 | Los Angeles   |          2 |
|      5 | San Francisco |          8 |
+--------+---------------+------------+


In [40]:
#Query to Select all boxes with a value larger than $150.
mycursor.execute ("SELECT * FROM boxes WHERE Value>150")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+------------+---------+-------------+
| Code   | Contents   |   Value |   Warehouse |
|--------+------------+---------+-------------|
| 0MN7   | Rocks      |     180 |           3 |
| 4H8P   | Rocks      |     250 |           1 |
| 4RT3   | Scissors   |     190 |           4 |
| 7G3H   | Rocks      |     200 |           1 |
| 9J6F   | Papers     |     175 |           2 |
+--------+------------+---------+-------------+


In [41]:
#Query to Select all distinct contents in all the boxes
mycursor.execute ("SELECT DISTINCT contents FROM boxes")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+------------+
| contents   |
|------------|
| Rocks      |
| Scissors   |
| Papers     |
+------------+


In [42]:
#Query to Select the average value of all the boxes.
mycursor.execute ("SELECT Avg (Value) FROM boxes")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+---------------+
|   Avg (Value) |
|---------------|
|       147.727 |
+---------------+


In [43]:
#Query to Select the warehouse code and the average value of the boxes in each warehouse.

mycursor.execute ("SELECT Warehouse , Avg (Value) FROM boxes GROUP BY Warehouse")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+-------------+---------------+
|   Warehouse |   Avg (Value) |
|-------------+---------------|
|           1 |         162.5 |
|           2 |         162.5 |
|           3 |         115   |
|           4 |         165   |
|           5 |          90   |
+-------------+---------------+


In [44]:
#Query to Select only those warehouses where the average value of the boxes is greater than 150
mycursor.execute ("SELECT Warehouse , Avg (Value) FROM boxes GROUP BY Warehouse HAVING Avg(Value)>150")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+-------------+---------------+
|   Warehouse |   Avg (Value) |
|-------------+---------------|
|           1 |         162.5 |
|           2 |         162.5 |
|           4 |         165   |
+-------------+---------------+


In [45]:
#Query to Select the code of each box, along with the name of the city the box is located in
mycursor.execute ("SELECT Boxes.Code,Location FROM Warehouses JOIN  Boxes ON Warehouses.Code")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+---------------+
| Code   | Location      |
|--------+---------------|
| 4H8P   | Chicago       |
| 4H8P   | Chicago       |
| 4H8P   | New York      |
| 4H8P   | Los Angeles   |
| 4H8P   | San Francisco |
| 7G3H   | Chicago       |
| 7G3H   | Chicago       |
| 7G3H   | New York      |
| 7G3H   | Los Angeles   |
| 7G3H   | San Francisco |
| 8JN6   | Chicago       |
| 8JN6   | Chicago       |
| 8JN6   | New York      |
| 8JN6   | Los Angeles   |
| 8JN6   | San Francisco |
| P0H6   | Chicago       |
| P0H6   | Chicago       |
| P0H6   | New York      |
| P0H6   | Los Angeles   |
| P0H6   | San Francisco |
| 9J6F   | Chicago       |
| 9J6F   | Chicago       |
| 9J6F   | New York      |
| 9J6F   | Los Angeles   |
| 9J6F   | San Francisco |
| P2T6   | Chicago       |
| P2T6   | Chicago       |
| P2T6   | New York      |
| P2T6   | Los Angeles   |
| P2T6   | San Francisco |
| 0MN7   | Chicago       |
| 0MN7   | Chicago       |
| 0MN7   | New York      |
| 0MN7   | Los Angeles   |
|

In [46]:
#Query to Select the warehouse codes, along with the number of boxes in each warehouse.
mycursor.execute ("SELECT Warehouse, Count(*) FROM Boxes GROUP BY Warehouse")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+-------------+------------+
|   Warehouse |   Count(*) |
|-------------+------------|
|           1 |          4 |
|           2 |          2 |
|           3 |          2 |
|           4 |          2 |
|           5 |          1 |
+-------------+------------+


In [47]:
#Query to Optionally, take into account that some warehouses are empty (i.e., the box count 
#should show up as zero, instead of omitting the warehouse from the result).
mycursor.execute ("SELECT Warehouses.Code, \
COUNT(Boxes.Code) FROM Warehouses \
LEFT JOIN Boxes ON Warehouses.Code = Boxes.Warehouse \
GROUP BY Warehouses.Code")

out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+---------------------+
|   Code |   COUNT(Boxes.Code) |
|--------+---------------------|
|      1 |                   4 |
|      2 |                   2 |
|      3 |                   2 |
|      4 |                   2 |
|      5 |                   1 |
+--------+---------------------+


In [48]:
#Query to Select the codes of all warehouses that are saturated (a warehouse is saturated if
#the number of boxes in it is larger than the warehouse's capacity).
mycursor.execute ("SELECT Code From Warehouses WHERE Capacity < (SELECT COUNT(*) FROM Boxes WHERE Warehouse=Warehouses.Code)")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+
|   Code |
|--------|
|      1 |
+--------+


In [49]:
#Query to Select the codes of all the boxes located in Chicago.
mycursor.execute ("SELECT Code FROM Boxes WHERE Warehouse IN (SELECT Code FROM Warehouses WHERE Location = 'Chicago')")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+
| Code   |
|--------|
| 4H8P   |
| 7G3H   |
| 8JN6   |
| P0H6   |
| 9J6F   |
| P2T6   |
+--------+


In [50]:
#Query to Create a new warehouse in New York with a capacity for 3 boxes.
mycursor.execute("INSERT INTO Warehouses (Code,Location,Capacity) VALUES (6,'New York',3)")
mydb.commit()

In [51]:
#Query to Create a new box, with code "H5RT", containing "Papers" with a value of $200,
#and located in warehouse 2.
mycursor.execute("INSERT INTO Boxes (Code,Contents,Value,Warehouse) VALUES ('H5RT','Papers',200,2)")
mydb.commit()

In [52]:
#Query to Reduce the value of all boxes by 15%.
mycursor.execute("""
UPDATE Boxes 
SET Value = Value * 0.85;
""")

mycursor.execute("SELECT * FROM Boxes ")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+------------+---------+-------------+
| Code   | Contents   |   Value |   Warehouse |
|--------+------------+---------+-------------|
| 0MN7   | Rocks      |  153    |           3 |
| 4H8P   | Rocks      |  212.5  |           1 |
| 4RT3   | Scissors   |  161.5  |           4 |
| 7G3H   | Rocks      |  170    |           1 |
| 8JN6   | Papers     |   63.75 |           1 |
| 8Y6U   | Papers     |   42.5  |           3 |
| 9J6F   | Papers     |  148.75 |           2 |
| H5RT   | Papers     |  170    |           2 |
| LL08   | Rocks      |  119    |           4 |
| P0H6   | Scissors   |  106.25 |           1 |
| P2T6   | Scissors   |  127.5  |           2 |
| TU55   | Papers     |   76.5  |           5 |
+--------+------------+---------+-------------+


In [53]:
#Query to Remove all boxes with a value lower than $100.
mycursor.execute(""" DELETE FROM Boxes WHERE Value < 100;""")

mycursor.execute("SELECT * FROM Boxes ")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+------------+---------+-------------+
| Code   | Contents   |   Value |   Warehouse |
|--------+------------+---------+-------------|
| 0MN7   | Rocks      |  153    |           3 |
| 4H8P   | Rocks      |  212.5  |           1 |
| 4RT3   | Scissors   |  161.5  |           4 |
| 7G3H   | Rocks      |  170    |           1 |
| 9J6F   | Papers     |  148.75 |           2 |
| H5RT   | Papers     |  170    |           2 |
| LL08   | Rocks      |  119    |           4 |
| P0H6   | Scissors   |  106.25 |           1 |
| P2T6   | Scissors   |  127.5  |           2 |
+--------+------------+---------+-------------+


In [54]:
#Query to Remove all boxes from saturated warehouses.
mycursor.execute("DELETE FROM Boxes WHERE Warehouse IN \
                (SELECT Code FROM Warehouses WHERE Capacity < \
                (SELECT COUNT(*) FROM Boxes WHERE Warehouse = Warehouses.Code))")

mycursor.execute("SELECT * FROM Boxes ")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------+------------+---------+-------------+
| Code   | Contents   |   Value |   Warehouse |
|--------+------------+---------+-------------|
| 0MN7   | Rocks      |  153    |           3 |
| 4H8P   | Rocks      |  212.5  |           1 |
| 4RT3   | Scissors   |  161.5  |           4 |
| 7G3H   | Rocks      |  170    |           1 |
| 9J6F   | Papers     |  148.75 |           2 |
| H5RT   | Papers     |  170    |           2 |
| LL08   | Rocks      |  119    |           4 |
| P0H6   | Scissors   |  106.25 |           1 |
| P2T6   | Scissors   |  127.5  |           2 |
+--------+------------+---------+-------------+


In [55]:
#Query to Add Index for column "Warehouse" in table "boxes"
mycursor.execute("CREATE INDEX indexwarehouse ON Boxes (Warehouse)")

mydb.commit()

mycursor.execute("SHOW INDEX FROM Boxes")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+---------+--------------+----------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------+
| Table   |   Non_unique | Key_name       |   Seq_in_index | Column_name   | Collation   |   Cardinality | Sub_part   | Packed   | Null   | Index_type   | Comment   | Index_comment   |
|---------+--------------+----------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------|
| boxes   |            0 | PRIMARY        |              1 | Code          | A           |             9 |            |          |        | BTREE        |           |                 |
| boxes   |            1 | indexwarehouse |              1 | Warehouse     | A           |             9 |            |          |        | BTREE        |           |                 |
+---------+--------------+----------------+----------------+---------------

In [56]:
#Query to Print all the existing indexes
mycursor.execute("SHOW INDEXES FROM Boxes")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+---------+--------------+----------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------+
| Table   |   Non_unique | Key_name       |   Seq_in_index | Column_name   | Collation   |   Cardinality | Sub_part   | Packed   | Null   | Index_type   | Comment   | Index_comment   |
|---------+--------------+----------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------|
| boxes   |            0 | PRIMARY        |              1 | Code          | A           |             9 |            |          |        | BTREE        |           |                 |
| boxes   |            1 | indexwarehouse |              1 | Warehouse     | A           |             9 |            |          |        | BTREE        |           |                 |
+---------+--------------+----------------+----------------+---------------

In [57]:
#To Find the Foreign Key constraint name in order to execute the query to remove the index previously added.
mycursor.execute("SELECT TABLE_NAME, CONSTRAINT_TYPE, CONSTRAINT_NAME\
                FROM information_schema.table_constraints\
                WHERE table_name='boxes';")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+--------------+-------------------+-------------------+
| TABLE_NAME   | CONSTRAINT_TYPE   | CONSTRAINT_NAME   |
|--------------+-------------------+-------------------|
| boxes        | PRIMARY KEY       | PRIMARY           |
| boxes        | FOREIGN KEY       | boxes_ibfk_1      |
+--------------+-------------------+-------------------+


In [58]:
#Query to Remove (drop) the index you added just
mycursor.execute ("ALTER TABLE Boxes DROP FOREIGN KEY boxes_ibfk_1")
mycursor.execute("DROP INDEX IF EXISTS indexwarehouse ON Boxes")

mydb.commit()

mycursor.execute("SHOW INDEX FROM Boxes")
out = mycursor.fetchall()
from tabulate import tabulate 
print (tabulate(out,headers = [i[0] for i in mycursor.description],tablefmt = 'psql'))

+---------+--------------+------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------+
| Table   |   Non_unique | Key_name   |   Seq_in_index | Column_name   | Collation   |   Cardinality | Sub_part   | Packed   | Null   | Index_type   | Comment   | Index_comment   |
|---------+--------------+------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------|
| boxes   |            0 | PRIMARY    |              1 | Code          | A           |             9 |            |          |        | BTREE        |           |                 |
+---------+--------------+------------+----------------+---------------+-------------+---------------+------------+----------+--------+--------------+-----------+-----------------+
